In [45]:
import numpy as np
import hashlib
import random
import string
import hmac
import pandas as pd
from tqdm.notebook import tqdm
import os
import re

import warnings
warnings.filterwarnings("ignore")

### Hashing Parameters and Functions

In [2]:
e = 2**52
salt = "0000000000000000000fa3b65e43e4240d71762a5bf397d5304b2596d116859c"
first_game = "77b271fe12fca03c618f63dfb79d4105726ba9d4a25bb3f1964e435ccf9cb209"
current_game = "948c8867a6582fdf55ef05b09030cb1afccf7d90f1d6844382577e2fc34e3419"

In [3]:
#functions to decode hash values to multiplier values
def get_result(game_hash):
    hm = hmac.new(str.encode(game_hash), b'', hashlib.sha256)
    hm.update(salt.encode("utf-8"))
    h = hm.hexdigest()
    if (int(h, 16) % 33 == 0):
        return 1
    h = int(h[:13], 16)
    e = 2**52
    return (((100 * e - h) / (e-h)) // 1) / 100.0

def get_prev_game(hash_code):
    m = hashlib.sha256()
    m.update(hash_code.encode("utf-8"))
    return m.hexdigest()

def get_all_multipliers(first_game,game_hash):

    results = []
    count = 0
    while game_hash != first_game:
        count += 1
        results.append(get_result(game_hash))
        game_hash = get_prev_game(game_hash)
    
    #append first game
    results.append(get_result(first_game))
    results = np.array(results)
    
    return results

### Load data

In [4]:
multipliers = get_all_multipliers(first_game,current_game)
multipliers_chronological_order = multipliers[::-1]
multiplier_df = pd.DataFrame({'game_no': range(1,len(multipliers_chronological_order)+1), 'multiplier': multipliers_chronological_order})

### Train Test Split


In [33]:
#train test split
train_pct = 0.8
train_test_split_index = round(len(multiplier_df.index) * train_pct)

testing_df = multiplier_df.loc[train_test_split_index:].copy()
training_df = multiplier_df.loc[:train_test_split_index-1].copy()

### Brute-Force Method
Adjust results directory per experiment run.

In [56]:
#experiment parameters
multiplier_checkout_level = 20
bet_size = 0.1 #USD
batch_starting_capital = 50 #USD
batch_size = 100

#make directory to store results in
experiment_results_directory = "brute_force_v1"
results_path = os.path.join("data",experiment_results_directory)
os.mkdir(results_path)
config_parameters = pd.Series({'multiplier_checkout_level':multiplier_checkout_level,
                               'bet_size':bet_size,
                               'batch_starting_capital':batch_starting_capital,
                               'batch_size':batch_size})
config_parameters.to_csv(os.path.join(results_path, "config_parameters.csv"))

In [57]:
#add columns
training_df["bet_size"] = np.NaN
training_df["leftover_capital"] = np.NaN
training_df["winnings"] = np.NaN

#create batches
training_batches = [training_df[i:i+batch_size] for i in range(0,training_df.shape[0],batch_size)]

In [ ]:
for batch_idx in tqdm(range(len(training_batches))):
    #extract batch
    batch = training_batches[batch_idx]
    #each batch run reset the starting capital
    starting_capital = batch_starting_capital #USD
    for index in batch.index:
        #place bet
        batch.loc[index,"bet_size"] = bet_size

        #calculate winnings
        if batch.loc[index, "multiplier"] < multiplier_checkout_level:
            batch.loc[index, "winnings"] = 0
        else:
            batch.loc[index, "winnings"] = multiplier_checkout_level * bet_size

        leftover_capital = starting_capital - bet_size + batch.loc[index, "winnings"]
        batch.loc[index,"leftover_capital"] = leftover_capital
        starting_capital = leftover_capital

        if starting_capital == 0:
            #end of experiment
            break

    #write out batch
    batch.to_csv(os.path.join(results_path,"batch_" + str(batch_idx) + ".csv"),index=False)

  0%|          | 0/15382 [00:00<?, ?it/s]

#### Analyse Brute-Force method

In [50]:
results_files = [file for file in os.listdir(results_path) if "csv" in file]
full_results = pd.DataFrame([])
for result in tqdm(results_files):
    batch_result = pd.read_csv(os.path.join(results_path, result))
    batch_num = re.sub(r'[^\d]+', '', result)
    #batch profit loss
    batch_capital_end = batch_result.iloc[-1]["leftover_capital"]
    if pd.isna(batch_capital_end):
        batch_profit_loss = 0
        perct_profit_loss = -1
    else:
        batch_profit_loss = batch_capital_end - batch_starting_capital
        perct_profit_loss = (batch_capital_end-batch_starting_capital)/batch_starting_capital

    #batch success
    if batch_profit_loss > 0:
        batch_success = True
    else:
        batch_success = False

    #batch number of wins
    number_of_wins = sum(batch_result["winnings"] > 0)
    #batch win %
    perct_win = number_of_wins/batch_size

    batch_info = pd.Series({'batch_num':batch_num,
               'starting_capital':batch_starting_capital,
               'ending_capital':batch_capital_end,
               'profit_loss':batch_profit_loss,
               'profit_loss_percentage':perct_profit_loss,
               'batch_success':batch_success,
               'num_of_batch_wins':number_of_wins,
               'batch_win_percentage':perct_win
               })
    full_results = full_results.append(batch_info, ignore_index=True)

full_results.to_csv(os.path.join(results_path,"brute_force_experiment_analysis_v1.csv"),index=False)

  0%|          | 0/15382 [00:00<?, ?it/s]